# Access to Green Space vs Depression

> A collection of visualizations to highlight how environmental and societal factors impact our mental well-being.

- toc: false
- badges: false
- comments: false

In [1]:
# hide
# Code Import

## Imports:

### Libraries
import numpy as np
import pandas as pd
import altair as alt
import os, os.path
import pycountry
from vega_datasets import data

alt.data_transformers.enable('default', max_rows=None)

import country_converter as coco
import geopandas as gpd
import json

### Data
cwd = os.getcwd()
df_file = '/depression_final_df.csv'
df = pd.read_csv(cwd + df_file)

countries = alt.topo_feature(data.world_110m.url, 'countries')

df = df.dropna(subset=['id'])
df.id = df.id.astype(int)
df = df.rename(columns={"Prevalence - Depressive disorders - Sex: Male - Age: Age-standardized (Percent)": "Depression in males (%)",
                        "Prevalence - Depressive disorders - Sex: Female - Age: Age-standardized (Percent)": "Depression in females (%)"})

sum_column = df["Depression in males (%)"] + df["Depression in females (%)"]

df['Depression (%)'] = sum_column

dfi = df.fillna(0)
dfi['Year'] = dfi['Year'].apply(int)
df['Year'] = pd.to_datetime(dfi.Year, format='%Y')

In [2]:
# hide
df_1990 = df[df['Year'] == '1990']
df_2000 = df[df['Year'] == '2000']
df_2010 = df[df['Year'] == '2010']
df_2017 = df[df['Year'] == '2017']

df_gender = dfi[['Country','Year','Depression in males (%)','Depression in females (%)']]
df_stats = df.drop(columns=['country_code','geometry','Continent',
                            'id','Population (historical estimates)',
                            'Depression in males (%)','Depression in females (%)','Social support',
                            'Time spent in social interactions','Life satisfaction',
                            'Access to green space']) 
df_gender = pd.melt(df_gender, id_vars=['Country','Year'], var_name=['Statistic'])
df_stats = pd.melt(df_stats, id_vars=['Country','Year'], var_name=['Statistic'])

df_europe = dfi.loc[dfi['Access to green space'] != 0]

In [3]:
#hide
click = alt.selection_multi(fields=['Country'], empty='none')

In [4]:
# hide

eu_click = alt.selection_multi(fields=['Country'], empty='all', bind='legend')
eu_color = alt.Color('Country:N', scale=alt.Scale(scheme = 'set1')),
eu_width = 300
eu_height = 400
eu_green_colour = alt.Color('Access to green space:Q',scale=alt.Scale(scheme='greens',reverse=False), legend=alt.Legend(orient="bottom"))
eu_colour = alt.Color('Depression (%):Q',scale=alt.Scale(scheme='viridis',reverse=True), legend=alt.Legend(orient="bottom"))

eu_scale = 300
eu_center = [-25,72]
eu_clip = [[0,0],[eu_width,eu_height]]

eu_green_choro = (alt.Chart(countries).mark_geoshape(
    stroke='black'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_europe, 'id', ['Depression (%)','Country', 'Access to green space'])
).encode(
    color=eu_green_colour,
    opacity=alt.condition(eu_click, alt.value(1), alt.value(0.2)),
    tooltip=['Country:N', 'Depression (%):Q']
).add_selection(
    eu_click
).project(
    type= 'mercator',
    scale= eu_scale,                          # Magnify
    center= eu_center,                     # [lon, lat]
    clipExtent= eu_clip, 
).properties(
    width=eu_width,
    height=eu_height
))

eu_choro = (alt.Chart(countries).mark_geoshape(
    stroke='black'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_europe, 'id', ['Depression (%)','Country', 'Access to green space'])
).encode(
    color=eu_colour,
    opacity=alt.condition(eu_click, alt.value(1), alt.value(0.2)),
    tooltip=['Country:N', 'Depression (%):Q']
).add_selection(
    eu_click
).project(
    type= 'mercator',
    scale= eu_scale,                          # Magnify
    center= eu_center,                     # [lon, lat]
    clipExtent= eu_clip, 
).properties(
    width=eu_width,
    height=eu_height
))


eu_back = alt.Chart(countries).mark_geoshape(
    fill='grey',
    stroke='black'
).project(
    type= 'mercator',
    scale= eu_scale,                          # Magnify
    center= eu_center,                     # [lon, lat]
    clipExtent= eu_clip,     # [[left, top], [right, bottom]]
).properties(
    width=eu_width, height=eu_height
)

In [5]:
# hide
eu_scatter = alt.Chart(df_europe).mark_circle(filled=True, size=200).encode(
    x=alt.X('Access to green space:Q', title = 'Access to green space', scale=alt.Scale(domain=[60, 100])),
    y=alt.Y('Depression (%):Q', title = 'Depression (%)', scale=alt.Scale(domain=[3, 11])),
    color = alt.Color('Country:N', scale=alt.Scale(scheme = 'set1'), legend=alt.Legend(orient="right")),
    opacity = alt.condition(eu_click, alt.value(1.0), alt.value(0.3)),
    tooltip = ['Country:N']
).add_selection(
    eu_click
).properties(
    width=2*eu_width,
    height=eu_height
)

eu_reg = eu_scatter.transform_regression('Access to green space','Depression (%)').mark_line()

## Green Space Access vs Depression

> Click on the country names to filter!

In [6]:
# hide_input
((eu_back + eu_green_choro) | (eu_back + eu_choro)).resolve_scale(color='independent') & (eu_scatter).resolve_scale(color='independent')# + eu_reg)

alt.VConcatChart(...)